<html><head><meta content="text/html; charset=UTF-8" http-equiv="content-type"><style type="text/css">ol</style></head><body class="c5"><p class="c0 c4"><span class="c3"></span></p><p class="c2 title" id="h.rrbabt268i6e"><h1>CaImAn&rsquo;s Demo pipeline</h1></p><p class="c0"><span class="c3">This notebook will help to demonstrate the process of CaImAn and how it uses different functions to denoise, deconvolve and demix neurons from a two-photon Calcium Imaging dataset. The demo shows how to construct the `params`, `MotionCorrect` and `cnmf` objects and call the relevant functions. You can also run a large part of the pipeline with a single method (`cnmf.fit_file`). See inside for details.

Dataset couresy of Sue Ann Koay and David Tank (Princeton University)

This demo pertains to two photon data. For a complete analysis pipeline for one photon microendoscopic data see demo_pipeline_cnmfE.ipynb</span></p>
<p class="c0"><span class="c3">More information can be found in the companion paper. </span></p>
</html>

In [ ]:
import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import matplotlib.animation as animation
try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

### Set up logger (optional)
You can log to a file using the filename parameter, or make the output more or less verbose by setting level to `logging.DEBUG`, `logging.INFO`, `logging.WARNING`, or `logging.ERROR`. A filename argument can also be passed to store the log file

In [ ]:
logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.WARNING)

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fname` variable as a list.

In [ ]:
directory = '/media/david/hdd1/David_GCAMP/20210827'
filename = 'file_00001_1'
fnames = [directory+'/sliced_data/'+filename+'.tif' ]
#fnames
#fnames = ['file_00011_cai1.tif']  # filename to be processed
#if fnames[0] in ['Sue_2x_3000_40_-46.tif', 'demoMovie.tif','file_00011_cai1.tif']:
#    fnames = [download_demo(fnames[0])]

In [ ]:
fnames

In [ ]:
#Read tag file
dict_name = directory+'/tag/'+filename[:-2]+'.json'
with open(dict_name, 'r') as f:
    tag = json.load(f)
length = tag['length']
scanZoomFactor = tag['scanZoomFactor']

### Play the movie (optional)
Play the movie (optional). This will require loading the movie in memory which in general is not needed by the pipeline. Displaying the movie uses the OpenCV library. Press `q` to close the video panel.

### Setup some parameters
We set some parameters that are relevant to the file, and then parameters for motion correction, processing with CNMF and component quality evaluation. Note that the dataset `Sue_2x_3000_40_-46.tif` has been spatially downsampled by a factor of 2 and has a lower than usual spatial resolution (2um/pixel). As a result several parameters (`gSig, strides, max_shifts, rf, stride_cnmf`) have lower values (halved compared to a dataset with spatial resolution 1um/pixel).

In [ ]:
# dataset dependent parameters
fr = 30                             # imaging rate in frames per second
decay_time = 0.4                    # length of a typical transient in seconds

# motion correction parameters
strides = (96, 96)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (32, 32)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (12,12)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
#p = 0 turns deconvolution off 
#If transients in your data rise instantaneously, set p = 1 (occurs at low sample rate or slow indicator). 
#If transients have visible rise time, set p = 2.
gnb = 2                     # number of global background components. This is a measure of the complexity of your background noise.
merge_thr = 0.7             # merging threshold, max correlation allowed
#Merging threshold of components after initialization. If two components are correlated more than this value, they are merged and treated as one.
rf = 50                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
#Should be at least 3 to 4 times larger than the expected neuron size to capture the complete neuron and its local background.
stride_cnmf = 30             # amount of overlap between the patches in pixels
#This should be roughly the neuron diameter. Larger overlap increases computational load, but yields better results during reconstruction/denoising of the data.
K = 4                       # Number of (expected) components per patch.
gSig = [16, 16]               # expected half size of neurons in pixels, CRUCIAL parameter for proper component detection.
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization
#Defaults to 1 (no compression). Can be set to 2 or even higher to save resources, but might impair quality.

# parameters for component evaluation
min_SNR = 2               # signal to noise ratio for accepting a component
#Peak SNR is calculated from strong calcium transients and the noise estimate.
rval_thr = 0.85              # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected
#CNN-based classifier (cnn): The shape of components is evaluated by a 4-layered convolutional neural network trained on a manually annotated dataset. 
#The CNN assigns a value of 0-1 to each component depending on its resemblance to a neuronal soma.

### Create a parameters object
You can creating a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values. The resulting `params` object is a collection of subdictionaries pertaining to the dataset to be analyzed `(params.data)`, motion correction `(params.motion)`, data pre-processing `(params.preprocess)`, initialization `(params.init)`, patch processing `(params.patch)`, spatial and temporal component `(params.spatial), (params.temporal)`, quality evaluation `(params.quality)` and online processing `(params.online)`

In [ ]:
opts_dict = {'fnames': fnames,
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr, 
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest}

opts = params.CNMFParams(params_dict=opts_dict)#Parameter object

### Setup a cluster
To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable `backend` determines the type of cluster used. The default value `'local'` uses the multiprocessing package. The `ipyparallel` option is also available. More information on these choices can be found [here](https://github.com/flatironinstitute/CaImAn/blob/master/CLUSTER.md). The resulting variable `dview` expresses the cluster option. If you use `dview=dview` in the downstream analysis then parallel processing will be used. If you use `dview=None` then no parallel processing will be employed.

## Memory mapping 

The cell below memory maps the file in order `'C'` and then loads the new memory mapped file. The saved files from motion correction are memory mapped files stored in `'F'` order. Their paths are stored in `mc.mmap_file`.

In [ ]:
fname_new = [directory+'/sliced_data/'+filename+'_memmap__d1_512_d2_512_d3_1_order_C_frames_'+str(length)+'_.mmap' ]
# now load the file
Yr, dims, T = cm.load_memmap(fname_new[0])
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)
    

Now start the cluster

In [ ]:
#%% restart cluster to clean up memory
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

## Run CNMF on patches in parallel

- The FOV is split is different overlapping patches that are subsequently processed in parallel by the CNMF algorithm.
- The results from all the patches are merged with special attention to idendtified components on the border.
- The results are then refined by additional CNMF iterations.

In [ ]:
%%capture
#%% RUN CNMF ON PATCHES
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0). If you want to have
# deconvolution within each patch change params.patch['p_patch'] to a
# nonzero value
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)

### Inspecting the results
Briefly inspect the results by plotting contours of identified components against correlation image.
The results of the algorithm are stored in the object `cnm.estimates`. More information can be found in the definition of the `estimates` object and in the [wiki](https://github.com/flatironinstitute/CaImAn/wiki/Interpreting-Results).

In [ ]:
#%% plot contours of found components
Cn = cm.local_correlations(images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn)

## Re-run (seeded) CNMF  on the full Field of View  
You can re-run the CNMF algorithm seeded on just the selected components from the previous step. Be careful, because components rejected on the previous step will not be recovered here.

In [ ]:
%%capture
#%% RE-RUN seeded CNMF on accepted patches to refine and perform deconvolution 
cnm2 = cnm.refit(images, dview=dview)

## Component Evaluation

The processing in patches creates several spurious components. These are filtered out by evaluating each component using three different criteria:

- the shape of each component must be correlated with the data at the corresponding location within the FOV
- a minimum peak SNR is required over the length of a transient
- each shape passes a CNN based classifier

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier
cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)

Plot contours of selected and rejected components

In [ ]:
#%% PLOT COMPONENTS
cnm2.estimates.plot_contours_nb(img=Cn, idx=cnm2.estimates.idx_components)
idx = cnm2.estimates.idx_components
empty_image = np.zeros((Cn.shape[0],Cn.shape[1]))
coor_g = [cnm2.estimates.coordinates[cr] for cr in idx]
#a = cm.utils.visualization.plot_only_contours(cnm2.estimates.A[:,idx],empty_image,coordinates=coor_g)

In [ ]:
cnm2.estimates.plot_only_accept(img=Cn, idx=cnm2.estimates.idx_components,display_numbers=True)

In [ ]:
#cnm2.estimates.plot_only_reject(img=Cn, idx=cnm2.estimates.idx_components,display_numbers=True)

In [ ]:
print('Origial component is ')
print(cnm2.estimates.idx_components)
cnm2.estimates.threshold_spatial_components()
A_thr = cnm2.estimates.A_thr
A_gt_thr_bin = A_thr.toarray() > 0
size_neurons_gt = A_gt_thr_bin.sum(0)
print('The size of spatial component is ')
print(np.sort(size_neurons_gt[idx]))

In [ ]:
min_size = 200
max_size = 2000
remove_small = 1
if remove_small:
    cnm2.estimates.remove_small_large_neurons(min_size,max_size)
print('After remove small and large')
print(cnm2.estimates.idx_components)
components_to_keep = cnm2.estimates.idx_components

In [ ]:
cnm2.estimates.plot_only_accept(img=Cn, idx=cnm2.estimates.idx_components,display_numbers=True)

In [ ]:
components_to_delete = [4,10]
components_to_delete = np.array(components_to_delete)-1
components_to_keep = np.delete(components_to_keep,components_to_delete)
cnm2.estimates.idx_components = np.intersect1d(cnm2.estimates.idx_components, components_to_keep)
cnm2.estimates.idx_components_bad = np.setdiff1d(np.arange(cnm2.estimates.A.shape[-1]), cnm2.estimates.idx_components)
print('After delete manually')
print(cnm2.estimates.idx_components)

In [ ]:
cnm2.estimates.plot_only_accept(img=Cn, idx=cnm2.estimates.idx_components,display_numbers=True)

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components)

In [ ]:
# rejected components
plot_reject = 0
if plot_reject:
    if len(cnm2.estimates.idx_components_bad) > 0:
        cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components_bad)
    else:
        print("No components were rejected.")

### Select only high quality components

In [ ]:
idx = cnm2.estimates.idx_components
cnm2.estimates.select_components(use_object=True)
cnm2.estimates.idx_components = idx

## Display final results

In [ ]:
cnm2.estimates.nb_view_components(img=Cn, denoised_color='red')

## Closing, saving, and creating denoised version
### You can save an hdf5 file with all the fields of the cnmf object

In [ ]:
save_results = 1
if save_results:
    if not os.path.isdir(directory+'/preprocess'):
        os.mkdir(directory+'/preprocess')
    if os.path.isfile(directory+'/preprocess/'+filename+'.hdf5'):
        os.remove(directory+'/preprocess/'+filename+'.hdf5')
    cnm2.save(directory+'/preprocess/'+filename+'.hdf5')

In [ ]:
#%% reconstruct denoised movie
denoised = cm.movie(cnm2.estimates.A.dot(cnm2.estimates.C) + \
                    cnm2.estimates.b.dot(cnm2.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])
if not os.path.isdir(directory+'/denoise'):
    os.mkdir(directory+'/denoise')
np.save(directory+'/denoise/'+'denoise_'+filename,denoised)

In [ ]:
save_movie = 0
coor_g = [cnm2.estimates.coordinates[cr] for cr in range(len(idx))]
if save_movie:
    # Use Agg backend for canvas
    from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(directory+'/movie/ROI_'+filename+'.avi', fourcc, 30.0, (432,288),0)
    # loop over your images
    for i in range(length):
        fig = plt.figure(1)
        img = denoised[i,:,:]
        #img = cv2.normalize(denoised[i,:,:].reshape(denoised.shape[2],denoised.shape[1]), None, 0, 255, cv2.NORM_MINMAX)
        plt.imshow(img,cmap=plt.cm.Greys_r)
        for c in coor_g:
            v = c['coordinates']
            plt.plot(*v.T,c='w')
        # put pixel buffer in numpy array
        canvas = FigureCanvas(fig)
        plt.close(1)
        canvas.draw()
        mat = np.array(canvas.renderer._renderer)
        mat = cv2.cvtColor(mat, cv2.COLOR_RGB2GRAY)
        # write frame to video
        out.write(mat)
    # close video writer
    #cv2.destroyAllWindows()
    out.release()

### Stop cluster and clean up log files

In [ ]:
#%% STOP CLUSTER and clean up log files

cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)
